In [1]:
from crewai import Agent, Task, Crew

llm = "gpt-4o-mini"

In [2]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

In [3]:
# Define the agent
data_analysis_agent = Agent(
        role="Data Analyser",
        goal="""You aim is to read and analyse sales data""",
        backstory="You are assigned to perform sales analysis for a company",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=True
    )

In [ ]:
def create_charts(query):
    create_chart = Task(
        description=query,
        expected_output="""A markdown summary and a matplotlib chart""",
        agent=data_analysis_agent,
        tools=[file_read_tool, file_writer_tool]
    )
    
    # Define the crew
    crew = Crew(
        agents=[data_analysis_agent],
        tasks=[create_chart],
        verbose=True
    )
    return crew, create_chart


files = [
    {
        'data_file_name':'./sales_product_cat.csv',
        'chart_file_name': './reports/sales_product_summary.png',
        'report_file_name': './reports/sales_product_summary.md',
    },
    {
        'data_file_name': './monthly_sales.csv',
        'chart_file_name': './reports/monthly_sales.png',
        'report_file_name': './reports/monthly_sales.md',
    },
    {
        'data_file_name': './sales_by_region.csv',
        'chart_file_name': './reports/sales_by_region.png',
        'report_file_name': './reports/sales_by_region.md',
    }
]
"""
data_file_name[0] ='./sales_product_cat.csv'
chart_file_name[0] ='./reports/sales_product_summary.png'
report_file_name[0] ='./reports/sales_product_summary.md'
data_file_name[1] ='./monthly_sales.csv'
chart_file_name[1] ='./reports/monthly_sales.png'
report_file_name[0] ='./reports/monthly_sales.md'
data_file_name[2] ='./sales_by_region.csv'
chart_file_name[2] ='./reports/sales_by_region.png'
report_file_name[0] ='./reports/sales_by_region.md'
"""

for f in files:
    query = f"""Read the Sales by product category data from the file '{f['data_file_name']}',
                draw a chart that summarizes the data and save it to the file '{f['chart_file_name']}'
                and write a summary of the data in markdown that includes a reference to the chart in the file '{f['report_file_name']}'"""
    print(query)
    #crew, task = create_charts(query)
    #crew.kickoff()
    #task_output = task.output


Read the Sales by product category data from the file './sales_product_cat.csv',
                draw a chart that summarizes the data and save it to the file './reports/sales_product_summary.png'
                and write a summary of the data in markdown that includes a reference to the chart in the file './reports/sales_product_summary.md'
Read the Sales by product category data from the file './monthly_sales.csv',
                draw a chart that summarizes the data and save it to the file './reports/monthly_sales.png'
                and write a summary of the data in markdown that includes a reference to the chart in the file './reports/monthly_sales.md'
Read the Sales by product category data from the file './sales_product_cat.csv',
                draw a chart that summarizes the data and save it to the file './reports/sales_product_summary.png'
                and write a summary of the data in markdown that includes a reference to the chart in the file './reports/sales_produ

In [5]:


print(f"Task Description: {task_output.description}")
print(f"Task Summary: {task_output.summary}")
print(f"Raw Output: {task_output.raw}")

Task Description: Read the Sales by product category data from the file './sales_product_cat.csv'
            and draw a chart that summarizes the data and save it to the file './sales_product_summary.png'
Task Summary: Read the Sales by product category data from the file...
Raw Output: ```python
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
file_path = './sales_product_cat.csv'
data = pd.read_csv(file_path)

# Summarize the data by product category
summary = data.groupby('Product Category').agg({'Units Sold': 'sum', 'Revenue ($)': 'sum'}).reset_index()

# Create a bar chart for Units Sold and Revenue
fig, ax1 = plt.subplots(figsize=(10, 6))

# Bar chart for Units Sold
ax1.bar(summary['Product Category'], summary['Units Sold'], color='b', alpha=0.7, label='Units Sold')
ax1.set_xlabel('Product Category')
ax1.set_ylabel('Units Sold', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Line chart for Revenue
ax2 = ax1.twinx()  # instantiate a second axes that s